In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
(X_train, y_train) , (X_test, y_test) = keras.datasets.mnist.load_data()


In [3]:
X_train = X_train / 255
X_test = X_test / 255

In [4]:
X_train_flattened = X_train.reshape(len(X_train), 28*28)
X_test_flattened = X_test.reshape(len(X_test), 28*28)

In [5]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2725 - accuracy: 0.9228
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1197 - accuracy: 0.9651
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0826 - accuracy: 0.9752
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0638 - accuracy: 0.9803
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0493 - accuracy: 0.9850


In [6]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0721 - accuracy: 0.9762


[0.07212480157613754, 0.9761999845504761]

In [7]:
model.save("./saved_model/")

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


# (1) Post training quantization, before size = 72 KB

In [8]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
tflite_model = converter.convert()

In [9]:
len(tflite_model) #without quantization

320004

In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT] ##applying algo to quantization
tflite_quant_model = converter.convert()

In [12]:
len(tflite_quant_model)

84880

# (2) Quantization aware training


In [16]:
import tensorflow_model_optimization as tfmot


quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()

ModuleNotFoundError: No module named 'tensorflow_model_optimization'

In [17]:
q_aware_model.fit(X_train, y_train, epochs=1)

NameError: name 'q_aware_model' is not defined

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qaware_model = converter.convert()

NameError: name 'q_aware_model' is not defined